# Neural Network Enhancement of ORB/RANSAC Outputs

In [2]:
import pandas as pd
import numpy as np
from utils import * # ⚠️⚠️ The functions in utils.py are not correct.
import cv2 as cv
import os
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation # If the module is not found, run `pip install scipy` in this .ipynb in a separate code block.

## Obtaining Training Labels

### Conversion of Net Transformation to Incremental Transformations
⚠️⚠️ The conversion is not correct.

In [3]:
# Read the data, which contains net transformations.
PATH_TO_INPUT_FILE = "train_labels_one_chain_only.csv"
PATH_TO_OUTPUT_FILE = "nn_Y.csv"

df = pd.read_csv(PATH_TO_INPUT_FILE)

# Create a new CSV file.
with open(PATH_TO_OUTPUT_FILE, "w") as f:

    # Write the first row of data in df to the new CSV file.
    df.iloc[[0]].to_csv(f, index=False)

    # Iterate through the rows of the dataframe, and calculate the incremental transformations.
    for i in range(1, 100):
        # Analogy: When i = 5, we wish to calculate the transformation which brings image 5 to image 4.
        # 05 -> 04 can be calculated by applying 05 -> 00, then 00 -> 04.

        curr_transformation_row = df.iloc[i]   # 05 -> 00
        prev_transformation_row = df.iloc[i-1] # 04 -> 00

        chain_id = curr_transformation_row["chain_id"]

        curr_R = np.array([curr_transformation_row["qw"], curr_transformation_row["qx"], curr_transformation_row["qy"], curr_transformation_row["qz"]])
        curr_T = np.array([curr_transformation_row["x"], curr_transformation_row["y"], curr_transformation_row["z"]])

        prev_R = np.array([prev_transformation_row["qw"], prev_transformation_row["qx"], prev_transformation_row["qy"], prev_transformation_row["qz"]])
        prev_T = np.array([prev_transformation_row["x"], prev_transformation_row["y"], prev_transformation_row["z"]])

        # Calculate 00 -> 04.
        inv_prev_R, inv_prev_T = calculate_inverse_transformation(prev_R, prev_T)

        # Calculate 05 -> 04.
        R_incremental, T_incremental = compose_transformations(curr_R, curr_T, inv_prev_R, inv_prev_T)

        # Write the incremental transformations to the new CSV file.
        f.write(f"{chain_id},{i},{T_incremental[0]},{T_incremental[1]},{T_incremental[2]},{R_incremental[0]},{R_incremental[1]},{R_incremental[2]},{R_incremental[3]}\n")

### (Sanity Check) Conversion Back to Net Transformations
Hopefully, the file we generate `PATH_TO_SANITY_CHECK_FILE` has similar transformations as the `PATH_TO_INPUT_FILE`.

Currently, the rotational components are reverse-engineered from the net transformations correctly! 🎉

However, the translational components are not. 🫤

In [4]:
PATH_TO_SANITY_CHECK_FILE = "train_labels_sanity_check.csv"

df = pd.read_csv(PATH_TO_OUTPUT_FILE)

# Create a new CSV file.
with open(PATH_TO_SANITY_CHECK_FILE, "w") as f:

    # Write the first row of data in df to the new CSV file.
    df.iloc[[0]].to_csv(f, index=False)

    # Iterate through the rows of the dataframe, and calculate the net transformation.
    T_net = np.zeros((3))
    R_net = np.array([1, 0, 0, 0])

    for i in range(1, 100):
        curr_row = df.iloc[i]
        chain_id = curr_row["chain_id"]

        T_incremental = np.array([curr_row["x"], curr_row["y"], curr_row["z"]])
        R_incremental = np.array([curr_row["qw"], curr_row["qx"], curr_row["qy"], curr_row["qz"]])

        R_net, T_net = compose_transformations(R_incremental, T_incremental, R_net, T_net)

        # Write the net transformations to the new CSV file.
        f.write(f"{chain_id},{i},{T_net[0]},{T_net[1]},{T_net[2]},{R_net[0]},{R_net[1]},{R_net[2]},{R_net[3]}\n")

# Using ORB, BFMatcher, RANSAC to Compute Transformations
ORB (Oriented FAST and Rotated BRIEF) detects features.

BFMatcher (Brute-Force Matcher) matches features between two images.

RANSAC (RANdom SAmple Consensus) estimates the transformation between two images.

## Image Loading

In [5]:
CHAIN_ID = '0a998b28bd'
ABSOLUTE_PATH_TO_FOLDER = os.path.abspath(os.path.join('..', 'data', 'images', CHAIN_ID))

images = []
for i in range(0, 100):
    PATH_TO_IMAGE = os.path.join(ABSOLUTE_PATH_TO_FOLDER, f"{i:03}" + ".png")
    img = cv.imread(PATH_TO_IMAGE)
    images.append(img)


## Computing Transformations
The between-image transformations are computed.

We wish to throw these into a neural network to adjust their values, especially for the translational components.

In [6]:
# The camera intrinsic matrix K is given.
# This describes the focal length, optical center, and skew of the camera.
# Source: https://www.drivendata.org/competitions/261/spacecraft-pose-estimation/page/834/#camera-intrinsic-parameters
K = np.array([[5.2125371e+03, 0.0000000e+00, 6.4000000e+02],
              [0.0000000e+00, 6.2550444e+03, 5.1200000e+02],
              [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]])

# As i increases from 0 to 98, the overall transformation which brings image i back to image 0 is calculated.
returning_rotation = np.array([1, 0, 0, 0])
returning_translation = np.array([0, 0, 0])

PATH_TO_RESULTS_FILE = "results.csv"
PATH_TO_INCREMENTAL_RESULTS_FILE = "nn_X.csv"

with open(PATH_TO_RESULTS_FILE, "w") as f, open(PATH_TO_INCREMENTAL_RESULTS_FILE, "w") as f2:
    f.write("chain_id,i,x,y,z,qw,qx,qy,qz\n")
    f.write(f"{CHAIN_ID},0,0.0,0.0,0.0,1.0,0.0,0.0,0.0\n")
    f2.write("chain_id,i,x,y,z,qw,qx,qy,qz\n")
    f2.write(f"{CHAIN_ID},0,0.0,0.0,0.0,1.0,0.0,0.0,0.0\n")

    # For each pair of images (i, i+1),...
    for i in range(0, 99):

        ###################################
        # FEATURE DETECTION AND MATCHING  #
        ###################################

        # Create an SIFT object and use it to detect key points and descriptors.
        # Note that we are using the (i+1)th image as the "before" image and the ith image as the "after "image
        # because we are required to calculate the transformation required to return to the (i+1)th image.
        sift = cv.SIFT_create()
        kp1, des1 = sift.detectAndCompute(images[i+1], None)
        kp2, des2 = sift.detectAndCompute(images[i], None)

        # Create a BFMatcher object.
        bf = cv.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)

        # Apply the ratio test to obtain unambiguous (good) matches.
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

        # Draw the matches.
        img_matches = cv.drawMatches(images[i+1], kp1, images[i], kp2, good_matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # Save the matches in a folder.
        # Create a folder to save the matches if it doesn't already exist.
        # if not os.path.exists("results"):
        #     os.makedirs("results")
        # cv.imwrite("results/matches_{:03d}_{:03d}.png".format(i, i+1), img_matches)

        ###################################
        # TRANSFORMATION CALCULATION      #
        ###################################

        # Extract the matched keypoints.
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Extract the fundamental matrix, which describes the relative motion between two camera angles.
        F, mask = cv.findFundamentalMat(src_pts, dst_pts, cv.FM_RANSAC)

        # (Sometimes, the fundamental matrix cannot be found. F will be None, which causes errors later on.)
        if F is None:
            print(f"From i = {i+1:02} to i = {i:02}, F could not be calculated, so F was set as the identity matrix.")
            F = np.eye(3)

        # (When there are seven matches, cv.findFundamentalMat() returns three solutions as a matrix of shape (9, 3).
        # We arbitrarily choose the first solution.)
        if F.shape == (9, 3):
            F = F[:3]

        # Adjust for the effects of the camera's distortion on the image.
        E = K.T @ F @ K

        # Extract the rotation matrix R and translation T.
        _, R, T, mask = cv.recoverPose(E, src_pts, dst_pts, K)
        T = T.flatten() # Change the shape of T from (3, 1) to (3,)

        # Convert the rotation matrix to a quaternion.
        r = Rotation.from_matrix(R)
        q_xyzw = r.as_quat()
        q_wxyz = np.roll(q_xyzw, 1)

        # Write the results to the CSV file for the neural network's training.
        f2.write(f"{CHAIN_ID},{i+1},{T[0]},{T[1]},{T[2]},{q_wxyz[0]},{q_wxyz[1]},{q_wxyz[2]},{q_wxyz[3]}\n")

        # Print the results.
        # (Turn off scientific notation. Five decimal places. Always leave a space before the ' ' for the sign.)
        np.set_printoptions(suppress=True, precision=5, sign=' ')
        print(f"From i = {i+1:02} to i = {i:02}, ΔR = {q_wxyz}, ΔT = {T}")

        # Write the results to the CSV file.
        returning_rotation, returning_translation = compose_transformations(q_wxyz, T, returning_rotation, returning_translation) 
        f.write(f"{CHAIN_ID},{i+1},{returning_translation[0]},{returning_translation[1]},{returning_translation[2]},{returning_rotation[0]},{returning_rotation[1]},{returning_rotation[2]},{returning_rotation[3]}\n")


From i = 01 to i = 00, ΔR = [ 0.99366  0.00085  0.01564  0.11136], ΔT = [-0.03038  0.00004 -0.99954]
From i = 02 to i = 01, ΔR = [ 0.98798  0.06133 -0.09128  0.10867], ΔT = [ 0.36538  0.49945 -0.78552]
From i = 03 to i = 02, ΔR = [ 0.9989  -0.00261 -0.00469  0.04666], ΔT = [ 0.02803 -0.00023 -0.99961]
From i = 04 to i = 03, ΔR = [ 0.99945 -0.00366  0.01256  0.03062], ΔT = [ 0.03634 -0.01315 -0.99925]
From i = 05 to i = 04, ΔR = [-0.24371  0.84636 -0.46641 -0.08216], ΔT = [-0.08992 -0.03841  0.99521]
From i = 06 to i = 05, ΔR = [ 0.99947  0.00133 -0.00717  0.03166], ΔT = [-0.01786 -0.02004  0.99964]
From i = 07 to i = 06, ΔR = [ 0.99878 -0.00321  0.00684 -0.04872], ΔT = [ 0.00645  0.00563  0.99996]
From i = 08 to i = 07, ΔR = [ 0.23337 -0.52451  0.81868 -0.01411], ΔT = [-0.27094 -0.0724   0.95987]
From i = 09 to i = 08, ΔR = [ 0.99991  0.00136  0.00605 -0.01225], ΔT = [-0.00933 -0.00263 -0.99995]
From i = 10 to i = 09, ΔR = [ 0.99971  0.00222 -0.00485 -0.02328], ΔT = [ 0.0043   0.01253 

error: OpenCV(4.9.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/calib3d/src/five-point.cpp:572: error: (-215:Assertion failed) npoints >= 0 && points2.checkVector(2) == npoints && points1.type() == points2.type() in function 'recoverPose'
